# Part 1: Camera calibration (Intrinsic parameters)

In [2]:
import numpy as np
import cv2
import glob
import os

In [3]:
print(cv2.__version__)

3.3.0


In [4]:
from platform import python_version
print(python_version())

3.6.3


In [5]:
 # termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

In [6]:
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.


In [7]:
images = glob.glob('./Chessboard3/*.JPG')


In [8]:
print(images)

['./Chessboard3/IMG_1068.JPG', './Chessboard3/IMG_1069.JPG', './Chessboard3/IMG_1057.JPG', './Chessboard3/IMG_1061.JPG', './Chessboard3/IMG_1060.JPG', './Chessboard3/IMG_1062.JPG', './Chessboard3/IMG_1063.JPG', './Chessboard3/IMG_1067.JPG', './Chessboard3/IMG_1066.JPG', './Chessboard3/IMG_1072.JPG', './Chessboard3/IMG_1064.JPG', './Chessboard3/IMG_1070.JPG', './Chessboard3/IMG_1059.JPG', './Chessboard3/IMG_1071.JPG', './Chessboard3/IMG_1065.JPG']


In [9]:
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv2.drawChessboardCorners(img, (9,6), corners,ret)
        name = fname.replace("./Chessboard/","")
        cv2.imwrite("./res/part 1/"+str(name),img)
        #print("./res/part 1/"+str(name)+".png")
        #cv2.imshow('img',img)
        #cv2.waitKey(500)
        #cv2.destroyAllWindows()

In [10]:
 ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

In [11]:
print(ret)


1.628830895986753


In [12]:
print(mtx)

[[2.35823484e+03 0.00000000e+00 1.27505685e+03]
 [0.00000000e+00 2.35363433e+03 9.73403504e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [13]:
print(dist)

[[ 0.10990381 -0.01088416 -0.00567028 -0.00663648 -1.30729001]]


In [14]:
[ 0.10990381 -0.01088417 -0.00567028 -0.00663648 -1.30729001]

[-1.22057713]

In [15]:
principalPoint = (mtx[0][2],mtx[1][2])

In [16]:
focal = (mtx[0][0]+mtx[1][1])/2

In [17]:
print(principalPoint)

(1275.0568472391526, 973.4035044444332)


In [18]:
img = cv2.imread('./Chessboard3/IMG_1068.JPG')

In [19]:
dstchess = cv2.undistort(img, mtx, dist)
cv2.imwrite('./calibresult/mytest.png',dstchess)

True

# Part 2: Take the pictures

# Part 3: Compute the relative camera pose

In [20]:
from matplotlib import pyplot as plt

imgo1 = cv2.imread('./Pair image2/myleft.JPG')  #queryimage # left image
imgo2 = cv2.imread('./Pair image2/myright.JPG') #trainimage # right image


## 3.1 undistort the picture

In [21]:
# undistort
dst1 = cv2.undistort(imgo1, mtx, dist)
cv2.imwrite('./calibresult/myleft.png',dst1)
# undistort
dst2 = cv2.undistort(imgo2, mtx, dist)
cv2.imwrite('./calibresult/myright.png',dst1)

True

In [22]:

img1 = cv2.imread('./calibresult/0.JPG')  #queryimage # left image
img1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
img2 = cv2.imread('./calibresult/1.JPG') #trainimage # right image
img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

In [23]:
# Initiate SIFT detector
sift = cv2.xfeatures2d.SIFT_create()
# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

# FLANN parameters
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(des1,des2,k=2)

good = []
pts1 = []
pts2 = []

# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
    if m.distance < 0.5*n.distance:
        good.append(m)
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)
        
pts1 = np.int32(pts1)
pts2 = np.int32(pts2)
F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_LMEDS)


In [24]:
print(pts1)

[[1926  462]
 [1130  439]
 [1180  441]
 ...
 [1370 1346]
 [ 658 1699]
 [1207  724]]


Got the list of best matches from both images

In [25]:

# We select only inlier points
pts1 = pts1[mask.ravel()==1]
pts2 = pts2[mask.ravel()==1]

In [26]:
print(F)

[[-3.84128923e-08  2.54493575e-06 -6.20588721e-04]
 [-7.91415426e-07  6.66958672e-08 -1.59205797e-02]
 [ 1.21834113e-05  1.26101212e-02  1.00000000e+00]]


Next we find the epilines. Epilines corresponding to the points in first image is drawn on second image. So mentioning of correct images are important here. We get an array of lines. So we define a new function to draw these lines on the images.

In [27]:
def drawlines(img1,img2,lines,pts1,pts2):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    r,c = img1.shape
    img1 = cv2.cvtColor(img1,cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2,cv2.COLOR_GRAY2BGR)
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        img1 = cv2.line(img1, (x0,y0), (x1,y1), color,1)
        img1 = cv2.circle(img1,tuple(pt1),5,color,-1)
        img2 = cv2.circle(img2,tuple(pt2),5,color,-1)
    return img1,img2

Now we find the epilines in both the images and draw them.

In [28]:
# Find epilines corresponding to points in right image (second image) and
# drawing its lines on left image
lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1,1,2), 2,F)
lines1 = lines1.reshape(-1,3)
img5,img6 = drawlines(img1,img2,lines1,pts1,pts2)

# Find epilines corresponding to points in left image (first image) and
# drawing its lines on right image
lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1,F)
lines2 = lines2.reshape(-1,3)
img3,img4 = drawlines(img2,img1,lines2,pts2,pts1)

plt.subplot(121),plt.imshow(img5)
plt.subplot(122),plt.imshow(img3)
cv2.imwrite("./res/part 2/epilineleft.png", img5)
cv2.imwrite("./res/part 2/epilineright.png", img3)
plt.show()
cv2.destroyAllWindows()

Compute the essential matrix E
https://docs.opencv.org/3.1.0/d9/d0c/group__calib3d.html#ga13f7e34de8fa516a686a56af1196247f

In [29]:
E, mask = cv2.findEssentialMat(pts1, pts2, focal, principalPoint, method=cv2.RANSAC, prob=0.999, threshold=3.0)

In [30]:
print(E)

[[ 0.00344321  0.27292553  0.05371569]
 [-0.10668505  0.00622317 -0.69677576]
 [-0.01352015  0.65216834 -0.01382731]]


In [31]:
R1,R2,t = cv2.decomposeEssentialMat(E)

### decompose the essential matrix, we got two rotaion matrixs, R1(3*3) and R2(3*3), and one translation vector (3*1).
### We need the  projection matrix of the first camera and projection matrix of the second camera to calculate the triangulatePoints
### projMatr1 – 3x4 projection matrix of the first camera.
### projMatr2 – 3x4 projection matrix of the second camera.

In [32]:
print(R1)

[[ 0.84899521  0.09170774 -0.52038143]
 [ 0.09331245 -0.99536707 -0.02317731]
 [-0.52009608 -0.02888064 -0.85361934]]


In [33]:
print(R2)

[[ 9.68320962e-01 -5.99263511e-02  2.42411525e-01]
 [ 6.19095553e-02  9.98081604e-01 -5.64872089e-04]
 [-2.41912633e-01  1.55545672e-02  9.70173352e-01]]


In [34]:
print(t)

[[ 0.91936328]
 [ 0.07852536]
 [-0.38549309]]


### There are 4 possible combination for the projection matrix.
#### Case 1:  projMatr1 = K [ I 0]   projMatr2 = K[R1 t]
#### Case 2:  projMatr1 = K[ I 0]   projMatr2 = K[R1  -t]
#### Case 3:  projMatr1 = K[ I 0]   projMatr2 = K[R2 t]
#### Case 4:  projMatr1 = K[ I 0]   projMatr2 = K[R2 -t]


In [35]:
points1 = np.array(pts1).astype(float).T 
points2 = np.array(pts2).astype(float).T

In [36]:
# make a projection matrix for camera 1
projMatr1 = mtx@np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0]])

In [37]:
# case 1
projMatr21 = mtx@np.concatenate((R1,t),1)
# case 2
projMatr22 = mtx@np.concatenate((R1,-t),1)
# case 3
projMatr23 = mtx@np.concatenate((R2,t),1)
# case 4
projMatr24 = mtx@np.concatenate((R2,-t),1)

In [38]:
triangulate1 = cv2.triangulatePoints(projMatr1, projMatr21,points1,points2)
triangulate2 = cv2.triangulatePoints(projMatr1, projMatr22,points1,points2)
triangulate3 = cv2.triangulatePoints(projMatr1, projMatr23,points1,points2)
triangulate4 = cv2.triangulatePoints(projMatr1, projMatr24,points1,points2)

## 3.2 check the distance
### 3.2.1 case 1

In [39]:
triangulate1world = np.true_divide(triangulate1.T[:,:3], triangulate1.T[:,[-1]]).T

In [40]:
print(triangulate1world.T)

[[ -4.9881863    3.92911773 -18.07132373]
 [ -0.09833085  -0.36255284   1.59871818]
 [ -0.06923101  -0.38847826   1.7175466 ]
 ...
 [ -0.19436969   0.05058088   1.15145954]
 [  0.25678797   0.3901072    2.94078597]
 [  0.09246336   0.36448552   2.29745015]]


In [41]:
print(triangulate1world[2].min())

-1470.0314412191167


#### case 1 got negtive distance, get rid of it.

### 3.2.2 case 2

In [42]:
triangulate2world = np.true_divide(triangulate2.T[:,:3], triangulate2.T[:,[-1]]).T
print(triangulate2world[2].min())

-902.423228173481


#### case 2 got negtive distance, get rid of it.

### 3.2.2 case 3

In [43]:
triangulate3world = np.true_divide(triangulate3.T[:,:3], triangulate3.T[:,[-1]]).T
print(triangulate3world[2].min())

-5.979697809617511


#### case 3 got negtive distance, so I get rit of this case.

### 3.2.2 case 4

In [44]:
triangulate4world = np.true_divide(triangulate4.T[:,:3], triangulate4.T[:,[-1]]).T
print(triangulate4world[2].min())

3.3684935610743167


#### case 4 has the positive distance , so the (R2,-t) is the right R and t

In [45]:
pixelh = mtx@triangulate4world

In [46]:
print(pixelh.T)

[[1.06748020e+04 2.55879846e+03 5.54248675e+00]
 [5.83287275e+03 2.26939254e+03 5.16178252e+00]
 [6.13372876e+03 2.29262796e+03 5.19807117e+00]
 ...
 [3.74847595e+03 4.60253533e+03 4.27429877e+00]
 [6.89192088e+03 5.98281421e+03 4.65363157e+00]
 [6.28557861e+03 6.17929780e+03 4.58812457e+00]]


In [47]:
pixel = np.true_divide(pixelh.T[:,:2],pixelh.T[:,[-1]])

In [48]:
print(points1.T)

[[1926.  462.]
 [1130.  439.]
 [1180.  441.]
 ...
 [ 877. 1076.]
 [1481. 1285.]
 [1370. 1346.]]


In [49]:
print(pixel)

[[1925.99504319  461.66974762]
 [1130.01133377  439.65287915]
 [1180.00092092  441.05359219]
 ...
 [ 876.98033082 1076.79307835]
 [1480.97690391 1285.62266192]
 [1369.96685853 1346.8025348 ]]


In [50]:
print(pixel.shape)

(944, 2)


In [51]:
imgleft = cv2.imread('./Pair image2/myleft.JPG')  # left image
imgright = cv2.imread('./Pair image2/myright.JPG') # right image



In [52]:
cv2.imshow('left',imgleft)

In [53]:
for point in pixel.astype(int):
    #print(tuple(point))
    
    imgleft = cv2.circle(imgleft,tuple(point),10,(0, 0, 200),-1)
for point in points1.T.astype(int):
    #print(tuple(point))
    imgleft = cv2.circle(imgleft,tuple(point),5,(200, 0, 0),-1)
cv2.imwrite("./res/part 3/reporject.png", imgleft)
#plt.imshow(imgleft)
#cv2.imshow('left',imgleft )
#cv2.waitKey(0)
#cv2.destroyAllWindows()


True

# Part 4: Plane-sweeping stereo

### 4.1 compute 20 depth to generate a array of 20 depth

In [54]:
dmin = triangulate4world[2].min()

In [55]:
dmax = triangulate4world[2].max()

In [56]:
ddepth = (dmax - dmin)/19

In [57]:
n = [0,0,1]

In [58]:
print("dmin: ", dmin, "dmax: ", dmax, "ddepth: ", ddepth)

dmin:  3.3684935610743167 dmax:  5.979697809617511 ddepth:  0.13743180255490495


In [59]:
def darray(dmin, dmax,number):
    da = []
    ddepth = (dmax - dmin)/19
    for i in range(number):
        da.append(dmin+i*ddepth)
    return da

In [60]:
deptharray = darray(dmin,dmax,20)

In [61]:
print(deptharray)

[3.3684935610743167, 3.505925363629222, 3.643357166184127, 3.7807889687390315, 3.9182207712939365, 4.055652573848842, 4.193084376403746, 4.330516178958652, 4.467947981513556, 4.605379784068461, 4.742811586623366, 4.880243389178271, 5.017675191733176, 5.155106994288081, 5.292538796842986, 5.42997059939789, 5.567402401952796, 5.704834204507701, 5.842266007062606, 5.979697809617511]


In [62]:
from numpy.linalg import inv

In [63]:
pmatrix1 = np.concatenate((R2,t), axis=1)
pmatrix2 = np.append(n,deptharray[0]).reshape(1,4)
p = np.concatenate([pmatrix1,pmatrix2])

In [64]:
I = np.eye(3, dtype=int)
print(I.shape)
o = np.zeros(3).reshape(3,1)
print(o.shape)
#pmatrix21 = np.concatenate((I,o),axis = 1)
#p1 = np.concatenate([pmatrix21,pmatrix2])

(3, 3)
(3, 1)


In [65]:
def Homo(k,R2,t,n,depth):
    pmatrix11 = np.concatenate((R2,t), axis=1)
    pmatrix21 = np.concatenate((I,o),axis = 1)
    pmatrix2 = np.append(n,depth).reshape(1,4)
    p1 = np.concatenate([pmatrix21,pmatrix2])
    p2 = np.concatenate([pmatrix11,pmatrix2])
    H = p1@inv(p2)
    C = H[0:3,0:3]
    Homop = C@inv(k)
    Homo = k@Homop
    return Homo

In [66]:
Homo1 = Homo(mtx,R2,t,n,deptharray[0])

In [67]:
print(Homo1)

[[ 1.16183054e+00  6.15787649e-02 -2.85471064e+02]
 [ 3.71406350e-02  9.97855247e-01 -6.70409453e+01]
 [ 9.83784866e-05 -2.29691567e-07  8.03288324e-01]]


In [68]:
print(deptharray)

[3.3684935610743167, 3.505925363629222, 3.643357166184127, 3.7807889687390315, 3.9182207712939365, 4.055652573848842, 4.193084376403746, 4.330516178958652, 4.467947981513556, 4.605379784068461, 4.742811586623366, 4.880243389178271, 5.017675191733176, 5.155106994288081, 5.292538796842986, 5.42997059939789, 5.567402401952796, 5.704834204507701, 5.842266007062606, 5.979697809617511]


In [69]:
imgr4 = cv2.imread('./calibresult/0.JPG')
imgrg4 = cv2.cvtColor(imgr4,cv2.COLOR_BGR2GRAY)

In [70]:
count = 0
for d in deptharray:
    Hh = Homo(mtx,R2,t,n,d)
    imgwarpn = cv2.warpPerspective(imgrg4,Hh,(imgrg4.shape[1],imgrg4.shape[0]))
    count+=1
    cv2.imwrite("./res/part 4/warp/warp"+str(count)+".png", imgwarpn)

In [71]:
#imgwarpn = cv2.warpPerspective(imgrg4,Homo1,(imgrg4.shape[1],imgrg4.shape[0]))

In [72]:
#cv2.imwrite("./res/part 4/warp.png", imgwarpn)

### 4.2 compute the depth picture.
#### depth1 = wrap1 - original
#### depth2 = wrap2 - orignial


In [73]:
imgoriginal = cv2.imread('./calibresult/0.JPG')
imgoriginalg = cv2.cvtColor(imgr4,cv2.COLOR_BGR2GRAY)

In [81]:
imgwarpd = glob.glob('./res/part 4/warp/*.png')

In [94]:
print(imgwarpd[0])

./res/part 4/warp/warp20.png


In [84]:
count = 0
imgdepth = np.zeros(imgoriginalg.shape)


In [99]:
imgw = cv2.imread(imgwarpd[0])

In [107]:
print(imgw[:,:,0].shape)

(1936, 2592)


In [105]:
print(imgoriginalg.shape)

(1936, 2592)


In [102]:
imgdepth = cv2.absdiff(imgw,imgoriginalg)

error: /Users/travis/build/skvark/opencv-python/opencv/modules/core/src/arithm.cpp:659: error: (-209) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function arithm_op


In [108]:
for fname in imgwarpd:
    imgw = cv2.imread(fname)
    imgdepth = cv2.absdiff(imgw[:,:,0],imgoriginalg)
    count += 1
    cv2.imwrite("./res/part 4/Depth/"+str(count)+".png",imgdepth)